# Asyncronous in depth

A synchronous program is executed one step at a time. Even with conditional branching, loops and function calls, you can still think about the code in terms of taking one execution step at a time.
An asynchronous program behaves differently. It still takes one execution step at a time. The difference is that the system may not wait for an execution step to be completed before moving on to the next one.

### 1. Asyncronous example without generator etc

with simple program we can simulate how asyncrnous actually works. in this example we create baseclass named Task
Task has run & ready attribute. we use run to go to next operation & use ready to check wheter the Task is finished.

In [15]:
import time
from abc import ABC, abstractmethod

class Task:
    def __init__(self):
        self.ready = False
    
    def run():
        return NotImplementedError()

class Sleep(Task):
    def __init__(self, duration:int = None):
        self.threshold = time.time() + duration
        self.ready = False    
    def run(self):
        now = time.time()
        if now >= self.threshold:
            self.ready = True

In [16]:
from typing import Iterable, Set, List

def wait(ts: Iterable[Task]):
    orig:List[Task] = list(ts)
    pending:Set[Task] = set(orig)
    before = time.time()
    
    while pending:
        for task in list(pending):
            task.run()
            if task.ready:
                pending.remove(task)
                
    print(f'duration = {time.time() - before}')

In [17]:
wait([Sleep(3) for _ in range(100)]) # the duration is same even thought in total there are hundred of sleep function

duration = 3.000004291534424


### 2. Asyncronous example with generator

in this example we use the same kind of program as the first example. but this time it use generator instead of sleep class

In [12]:
from typing import Generator, List

def sleep_generator(duration):
    start = time.time()
    while (time.time() - start) < duration:
        yield
        
def wait_gen(ts: List[Generator]):
    orig:List[Task] = list(ts)
    pending:Set[Task] = set(orig)
    before = time.time()
    
    while pending:
        for task in list(pending):
            try:
                task.send(None)
            except StopIteration:    
                pending.remove(task)
                
    print(f'duration = {time.time() - before}')

In [14]:
wait_gen([sleep_generator(3) for _ in range(100)])

duration = 3.0000505447387695


In [18]:
start = time.time()
for _ in sleep_generator(2):
    print("loading", time.time() - start)
    time.sleep(0.3)

loading 4.673004150390625e-05
loading 0.30051279067993164
loading 0.6011500358581543
loading 0.9021131992340088
loading 1.2028493881225586
loading 1.5038261413574219
loading 1.8044207096099854


### 3. Asyncronous with multiple related generator

in this example, we handle more than 1 generator for each task. to show how generator can actually called each other & still have asyncronous behavior (pending & resume)

In [31]:
def bar():
    yield from sleep_generator(2) # generator function from example #2
    return 123

def foo():
    value = yield from bar()
    return value

wait_gen([foo() for _ in range(100)])

duration = 2.000169038772583


from example 3 above, u can see that it still return 2 seconds in duration even though the sleep generator is called from other generator.
python generator actually go to other generator process from foo yield to sleep generator yield & so on between each foo.